## 16.pytorch神经网络基础

### 自定义层：层也是nn.Module的子类
#### 构造一个没有任何参数的自定义层

In [1]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

#### 将层作为组件，用于构建更复杂的模型

In [2]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(2.3283e-09, grad_fn=<MeanBackward0>)

#### 带参数的图层

In [3]:
class MyLinear(nn.Module):
    '''自定义线性层'''
    def __init__(self, in_units, units):
        '''in_units为输入的维数，units为本层的神经元个数，也是输出的维数'''
        super().__init__()
        # 放入nn.Parameter中，完成“加梯度”和“赋予合适的名称”的过程
        self.weight = nn.Parameter(torch.randn(in_units, units))        # 初始化为标准正态分布
        self.bias = nn.Parameter(torch.randn(units,))                   # 为了表示为向量，所以加了一个逗号

    def forward(self, X):
        '''前向计算'''
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[ 0.4068, -0.6880,  0.2448],
        [-0.8242,  1.2040, -1.3183],
        [ 1.5116,  0.8371, -0.3327],
        [ 0.5534, -0.8892,  0.1395],
        [ 0.7824,  1.0587,  0.9352]], requires_grad=True)

使用自定义层直接执行正向传播计算

In [4]:
dense(torch.rand(2,5))

tensor([[1.0284, 2.8729, 0.3375],
        [1.8216, 2.9990, 0.6629]])

使用自定义层构建模型

In [5]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[7.3938],
        [8.5385]])